In [1]:
import numpy as np

import json

In [2]:
def pprint(a):
    print(json.dumps(a, sort_keys=False, ensure_ascii=False, indent=4, separators=(',', ': ')))

Запрос: отбор кандидатов -> отбор кандидат

Релевантные документы:
    1. кандидат отобрать претендент
    2. отбор выбрать претендент
    
df:
    * отбор       70000;
    * кандидат    70000;
    * претендент  30000;
    * отобрать    50000;
    * выбрать     70000;

In [3]:
docs = [
    "кандидат отобрать претендент",
    "отбор выбрать претендент",
]

query = "отбор кандидат"

In [4]:
alpha, beta = 0.7, 0.3

N = 1_000_000

In [5]:
dwords = ['отбор', 'кандидат', 'претендент', 'отобрать', 'выбрать']

tf_docs = [
    np.asarray([0, 1, 1, 1, 0], dtype=np.float32),
    np.asarray([1, 0, 1, 0, 1], dtype=np.float32),
]

v_query = np.asarray([1, 1, 0, 0, 0], dtype=float)

df = np.asarray([70000, 70000, 30000, 50000, 70000])

In [6]:
idf = np.log(N / df)

In [7]:
v_docs = [d * idf for d in tf_docs]

for d_i, d in enumerate(v_docs, 0):
    print("Вектор документа {}: \"{}\"".format(d_i + 1, docs[d_i]))
    pprint(dict(zip(dwords, d)))
    print()

Вектор документа 1: "кандидат отобрать претендент"
{
    "отбор": 0.0,
    "кандидат": 2.659260036932778,
    "претендент": 3.506557897319982,
    "отобрать": 2.995732273553991,
    "выбрать": 0.0
}

Вектор документа 2: "отбор выбрать претендент"
{
    "отбор": 2.659260036932778,
    "кандидат": 0.0,
    "претендент": 3.506557897319982,
    "отобрать": 0.0,
    "выбрать": 2.659260036932778
}



In [8]:
def norm(a):
    return np.sqrt(np.power(a, 2).sum())

In [9]:
v_docs_norm = [d / norm(d) for d in v_docs]

for d_i, d in enumerate(v_docs_norm, 0):
    print("Нормализованный вектор документа {}: \"{}\"".format(d_i + 1, docs[d_i]))
    pprint(dict(zip(dwords, d)))
    print()

Нормализованный вектор документа 1: "кандидат отобрать претендент"
{
    "отбор": 0.0,
    "кандидат": 0.49951136713259214,
    "претендент": 0.6586665105681685,
    "отобрать": 0.5627138011114365,
    "выбрать": 0.0
}

Нормализованный вектор документа 2: "отбор выбрать претендент"
{
    "отбор": 0.5171732176627488,
    "кандидат": 0.0,
    "претендент": 0.6819558093195759,
    "отобрать": 0.0,
    "выбрать": 0.5171732176627488
}



In [10]:
from functools import reduce

v_query_m = alpha * v_query + beta * reduce(lambda x, y: x + y, v_docs_norm) / len(v_docs_norm)

print("Исходный вектор запроса: \"{}\"".format(query))
pprint(dict(zip(dwords, v_query)))
print()

print("Модифицированный вектор запроса: \"{}\"".format(query))
pprint(dict(zip(dwords, v_query_m)))

Исходный вектор запроса: "отбор кандидат"
{
    "отбор": 1.0,
    "кандидат": 1.0,
    "претендент": 0.0,
    "отобрать": 0.0,
    "выбрать": 0.0
}

Модифицированный вектор запроса: "отбор кандидат"
{
    "отбор": 0.7775759826494123,
    "кандидат": 0.7749267050698888,
    "претендент": 0.20109334798316167,
    "отобрать": 0.08440707016671548,
    "выбрать": 0.07757598264941232
}
